# 🏆 TFT Leaderboard Crawler - VN2 Server

In [ ]:
# ========================================
# CELL 1: CONFIG
# ========================================

import requests
import time
import json
import pandas as pd
from datetime import datetime

# Riot API Key
API_KEY = "YOUR_RIOT_API_KEY_HERE"

PLATFORM = "vn2"
REGION_ACCOUNT = "asia"
REGION_MATCH = "sea"

TOP_N = 40
MATCH_COUNT_PER_PLAYER = 10

print("✅ Config loaded!")

In [ ]:
# ========================================
# CELL 2: API FUNCTIONS
# ========================================

def make_api_request(url, params=None):
    headers = {"X-Riot-Token": API_KEY}
    try:
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            return response.json()
        elif response.status_code == 429:
            retry_after = int(response.headers.get('Retry-After', 10))
            print(f"⏳ Rate limited. Waiting {retry_after}s...")
            time.sleep(retry_after)
            return make_api_request(url, params)
        return None
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

def format_timestamp(timestamp_ms):
    return datetime.fromtimestamp(timestamp_ms / 1000).strftime('%Y-%m-%d %H:%M:%S')

def format_unit_info(unit):
    """Format thông tin chi tiết của một unit"""
    name = unit.get('character_id', 'Unknown')
    tier = unit.get('tier', 1)  # Star level (1-3)
    rarity = unit.get('rarity', 0)  # Cost (0=1cost, 1=2cost, etc)
    items = unit.get('itemNames', [])  # Item names
    
    # Rarity to cost mapping
    cost = rarity + 1
    
    # Format items
    items_str = '+'.join(items) if items else 'No items'
    
    return {
        'name': name,
        'tier': tier,
        'cost': cost,
        'items': items_str,
        'item_count': len(items)
    }

print("✅ Functions loaded!")

In [ ]:
# ========================================
# CELL 3: FETCH LEADERBOARD
# ========================================

print("🏆 Fetching TFT Leaderboard...\n")

all_players = []

url = f"https://{PLATFORM}.api.riotgames.com/tft/league/v1/challenger"
data = make_api_request(url)
if data:
    entries = data.get('entries', [])
    for e in entries:
        e['tier'] = 'CHALLENGER'
        all_players.append(e)
    print(f"✅ Challenger: {len(entries)}")

if len(all_players) < TOP_N:
    url = f"https://{PLATFORM}.api.riotgames.com/tft/league/v1/grandmaster"
    data = make_api_request(url)
    if data:
        entries = data.get('entries', [])
        for e in entries:
            e['tier'] = 'GRANDMASTER'
            all_players.append(e)
        print(f"✅ Grandmaster: {len(entries)}")

all_players = sorted(all_players, key=lambda x: x.get('leaguePoints', 0), reverse=True)
top_players = all_players[:TOP_N]

print(f"\n🏆 Top {len(top_players)} selected!")

In [ ]:
# ========================================
# CELL 4: GET RIOT IDs
# ========================================

print("📋 Fetching Riot IDs...\n")

top_players_detailed = []

for i, player in enumerate(top_players, 1):
    puuid = player.get('puuid')
    print(f"⏳ [{i}/{len(top_players)}] ", end="")
    
    if puuid:
        account_url = f"https://{REGION_ACCOUNT}.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}"
        account_data = make_api_request(account_url)
        
        if account_data:
            game_name = account_data.get('gameName', 'Unknown')
            tag_line = account_data.get('tagLine', 'VN2')
            riot_id = f"{game_name}#{tag_line}"
            
            info = {
                'rank': i,
                'riot_id': riot_id,
                'game_name': game_name,
                'tag_line': tag_line,
                'tier': player.get('tier'),
                'lp': player.get('leaguePoints', 0),
                'wins': player.get('wins', 0),
                'losses': player.get('losses', 0),
                'puuid': puuid
            }
            info['total_games'] = info['wins'] + info['losses']
            info['win_rate'] = round(info['wins'] / info['total_games'] * 100, 1) if info['total_games'] > 0 else 0
            
            top_players_detailed.append(info)
            print(f"✅ {riot_id} ({info['lp']} LP)")
        else:
            print(f"❌ Failed")
    else:
        print(f"❌ No PUUID")
    
    time.sleep(0.3)

print(f"\n✅ Got {len(top_players_detailed)} players!")

In [ ]:
# ========================================
# CELL 5: DISPLAY LEADERBOARD
# ========================================

leaderboard_df = pd.DataFrame(top_players_detailed)

if len(leaderboard_df) > 0:
    print(f"\n🏆 TFT LEADERBOARD - VN2 - TOP {len(leaderboard_df)}\n")
    pd.set_option('display.max_rows', 50)
    display(leaderboard_df[['rank', 'riot_id', 'tier', 'lp', 'wins', 'losses', 'win_rate', 'total_games']])

In [ ]:
# ========================================
# CELL 6: FETCH MATCH HISTORY (với unit details)
# ========================================

if len(top_players_detailed) == 0:
    print("⚠️ No players to fetch.")
else:
    print(f"📊 Fetching match history ({MATCH_COUNT_PER_PLAYER} matches/player)...\n")
    
    all_player_stats = []  # Match stats
    all_unit_details = []  # Unit details per match
    
    for player in top_players_detailed:
        puuid = player['puuid']
        riot_id = player['riot_id']
        rank = player['rank']
        
        print(f"🎮 [{rank}] {riot_id} ", end="")
        
        match_url = f"https://{REGION_MATCH}.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids"
        match_ids = make_api_request(match_url, {"count": MATCH_COUNT_PER_PLAYER})
        
        if not match_ids:
            print("⚠️ No matches")
            continue
        
        matches_fetched = 0
        for match_id in match_ids:
            detail_url = f"https://{REGION_MATCH}.api.riotgames.com/tft/match/v1/matches/{match_id}"
            match_detail = make_api_request(detail_url)
            
            if match_detail:
                info = match_detail.get('info', {})
                for p in info.get('participants', []):
                    if p.get('puuid') == puuid:
                        # Traits
                        traits = [t['name'] for t in p.get('traits', []) if t.get('tier_current', 0) > 0]
                        
                        # Units với chi tiết
                        units = p.get('units', [])
                        unit_names = []
                        unit_details_str = []
                        
                        for unit in units:
                            unit_info = format_unit_info(unit)
                            unit_names.append(unit_info['name'])
                            
                            # Format: Name(tier★, cost$, items)
                            detail = f"{unit_info['name']}({unit_info['tier']}★{unit_info['cost']}$)"
                            if unit_info['item_count'] > 0:
                                detail += f"[{unit_info['items']}]"
                            unit_details_str.append(detail)
                            
                            # Lưu chi tiết riêng cho phân tích
                            all_unit_details.append({
                                'riot_id': riot_id,
                                'match_id': match_id,
                                'placement': p.get('placement'),
                                'unit_name': unit_info['name'],
                                'unit_tier': unit_info['tier'],
                                'unit_cost': unit_info['cost'],
                                'unit_items': unit_info['items'],
                                'item_count': unit_info['item_count']
                            })
                        
                        # Match stats
                        all_player_stats.append({
                            'riot_id': riot_id,
                            'leaderboard_rank': rank,
                            'match_id': match_id,
                            'datetime': format_timestamp(info.get('game_datetime', 0)),
                            'placement': p.get('placement'),
                            'level': p.get('level'),
                            'traits': ', '.join(traits[:5]),
                            'units': ', '.join(unit_names),
                            'units_detailed': ' | '.join(unit_details_str),
                            'num_units': len(units)
                        })
                        matches_fetched += 1
                        break
            time.sleep(0.25)
        
        print(f"✅ {matches_fetched} matches")
    
    print(f"\n✅ Total: {len(all_player_stats)} matches, {len(all_unit_details)} unit records")

In [ ]:
# ========================================
# CELL 7: DISPLAY & SAVE DATA
# ========================================

match_df = pd.DataFrame(all_player_stats) if 'all_player_stats' in dir() and all_player_stats else pd.DataFrame()
unit_df = pd.DataFrame(all_unit_details) if 'all_unit_details' in dir() and all_unit_details else pd.DataFrame()

print("💾 Saving...\n")

# Save leaderboard
if len(leaderboard_df) > 0:
    filename = f"tft_leaderboard_{PLATFORM}_top{TOP_N}.csv"
    leaderboard_df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ {filename}")

# Save match history
if len(match_df) > 0:
    filename = f"tft_matches_{PLATFORM}_top{TOP_N}.csv"
    match_df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ {filename}")

# Save unit details
if len(unit_df) > 0:
    filename = f"tft_units_{PLATFORM}_top{TOP_N}.csv"
    unit_df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"✅ {filename}")

print("\n📊 Match History Preview:")
if len(match_df) > 0:
    display(match_df[['riot_id', 'placement', 'level', 'units_detailed']].head(10))

print("\n📊 Unit Details Preview:")
if len(unit_df) > 0:
    display(unit_df.head(20))

print("\n🎉 Done!")

In [ ]:
# ========================================
# CELL 8: ANALYZE UNITS
# ========================================

if len(unit_df) > 0:
    print("📊 UNIT ANALYSIS\n")
    
    # Most used units
    print("🔥 Most Used Units:")
    unit_counts = unit_df['unit_name'].value_counts().head(15)
    for unit, count in unit_counts.items():
        print(f"   {unit}: {count}")
    
    # Most common 3-star units
    print("\n⭐ Most Common 3-Star Units:")
    three_star = unit_df[unit_df['unit_tier'] == 3]['unit_name'].value_counts().head(10)
    for unit, count in three_star.items():
        print(f"   {unit}: {count}")
    
    # Most common items
    print("\n🛡️ Most Common Items:")
    all_items = []
    for items_str in unit_df['unit_items'].dropna():
        if items_str and items_str != 'No items':
            items = items_str.split('+')
            all_items.extend(items)
    
    item_counts = pd.Series(all_items).value_counts().head(15)
    for item, count in item_counts.items():
        print(f"   {item}: {count}")
    
    # Units with most items
    print("\n🎯 Units with Most Items:")
    carry_units = unit_df[unit_df['item_count'] >= 3].groupby('unit_name').size().sort_values(ascending=False).head(10)
    for unit, count in carry_units.items():
        print(f"   {unit}: {count} times with 3+ items")